# Reflection and Blogpost Writing

## Setup

In [1]:
from utils import get_openai_api_key
OPENAI_API_KEY = get_openai_api_key()
#print(OPENAI_API_KEY)
llm_config = {"model": "gpt-3.5-turbo","api_key":OPENAI_API_KEY}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [5]:
reply = writer.generate_reply(messages=[{"content": task,"role": "user"}])

In [9]:
print(reply)

Title: "Discovering Hope AI: Transforming Lives Through Technology"

In a world where technological advancements continue to reshape our reality, Hope AI stands out as a beacon of innovation and compassion. This revolutionary artificial intelligence is not just a product of ones and zeros; it represents a promise of a better future. By harnessing the power of data and machine learning, Hope AI is making strides in healthcare, education, and beyond. Its ability to analyze complex information and provide personalized solutions is unlocking new possibilities for individuals worldwide. Join the movement towards a brighter tomorrow with Hope AI leading the way.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [10]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [11]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Potential of AI with Hope AI"

In a world where technology continues to revolutionize industries, Hope AI emerges as a beacon of innovation. This cutting-edge AI system offers a unique approach to problem-solving by infusing positivity and optimism into its algorithms. By prioritizing hope and resilience, Hope AI not only generates results but also fosters a sense of possibility and inspiration. In a landscape saturated with automation and efficiency, Hope AI stands out for its human-centric design and its commitment to uplifting solutions. Embrace the future of AI with Hope AI and witness the transformative power of positivity in technology.

--------------------------------------------------------------------------

## Nested chat

In [19]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO_Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [20]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal_Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [21]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics_Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [22]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta_Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [23]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [24]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [25]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
      Hope AI . Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Unlocking the Potential of AI with Hope AI"

In a world where technology continues to revolutionize industries, Hope AI emerges as a beacon of innovation. This cutting-edge AI system offers a unique approach to problem-solving by infusing positivity and optimism into its algorithms. By prioritizing hope and resilience, Hope AI not only generates results but also fosters a sense of possibility and inspiration. In a landscape saturated with automation and efficiency, Hope AI stands out for its human-centric design and its commitment to uplifting solutions. Embrace the future of AI with Hope AI and witness the transformative power of positivity in technology.

--------------------------------------------------------------------------

## Get the summary

In [26]:
print(res.summary)

Title: "Navigating the Future with Hope AI"

Amidst the rapid evolution of technology, Hope AI emerges as a promising force driving innovation. This advanced system reshapes problem-solving with a touch of positivity, offering not just solutions, but a sense of inspiration. While Hope AI showcases human-centric design, its true impact lies in instilling resilience and optimism. However, it's crucial to back claims with evidence, manage expectations, and address ethical implications. By embracing Hope AI responsibly, we embark on a journey that marries technological progress with human values. Let's navigate the future hand in hand with Hope AI, where innovation meets hope.
